In [ ]:
import sys
import pandas as pd

sys.path.append("../../")


In [ ]:
from monologue.core.agents import BasicToolUsingAgent,QuestionGeneratingAgent, BasicTypedResponseToolUsingAgent
from monologue.core.data.stores import VectorDataStore, ColumnarDataStore, EntityDataStore
from monologue.entities.examples import AvengingPassengersInstruct, NycTripEvent, Places, AbstractVectorStoreEntry, BookReviewers
from monologue.entities.examples import AbstractEntity
from monologue.core.data.stores import tools_for_entity
AbstractEntity.get_type('examples', 'Places')

In [ ]:
#these data were ingested from the logs
store = ColumnarDataStore(BookReviewers)
store("Who gave the Hobbit the highest review? Please tell me everything you know about that person")

#### You can create dynamic types 
- this is useful because we have assumed everything is driven by types but we might not always have one

In [ ]:
tools = [VectorDataStore(AvengingPassengersInstruct, extra_context="This stores inforamtion about people travelling in New York taxis").as_tool(),
         VectorDataStore(Places, extra_context="This stores inforamtion about places in New York").as_tool(),
         ColumnarDataStore(NycTripEvent).as_tool()
        ]

agent = BasicToolUsingAgent(tools=tools, context="Answer questions about people taking trips in new york")
agent("Please provide a summary of the person who went to Carroll Gardens most often with as much detail as possible. what might they have gont to Carroll gardens?")


In [ ]:
import monologue
#we use this to get fenced stuff out
from monologue.core.utils.ops import parse_fenced_code_blocks
#saves typing, fairly basic- generate pydantic types from example data
from monologue.core.utils.ops import pydantic_type_generator
#the entities we will create extend this which does the repr and pulls out some metadata
from monologue.entities import AbstractEntity

## Illustrate the type representation in logs

In [ ]:
from pydantic import Field
class MyEntity(AbstractEntity):
    code: str = Field(is_key=True)
    created_at: str

my_entity = MyEntity(code='test', created_at= "2023-01-01")
my_entity

## Show how the Columnar Store works

In [ ]:
from monologue.core.data.stores import ColumnarDataStore
from monologue.entities.examples import NycTripEvent
store = ColumnarDataStore(NycTripEvent)
store

### load the tool and ask questions

In [ ]:
tool = store.as_tool()
tool

In [ ]:
tool.run("What is least popular destination in new york city? Who has travelled there?")
#or just store("ask question")

# Vector Store Loading

### bios

In [ ]:
import pandas as pd
from monologue.core.data.stores import VectorDataStore
from monologue.entities.examples import *
from monologue import logger

In [ ]:
data = pd.read_csv("/Users/sirsh/Downloads/marvel_bios.csv").rename(columns={'entity_key':'id'})
data['id'] = data['id'].map(lambda x: x.replace('"',''))
data['name'] = data['id'].map(lambda x: x.split('_')[0])

data['doc_id'] = data['name']
data.tail()

In [ ]:
store = VectorDataStore(AvengingPassengersInstruct)
records = [AvengingPassengersInstruct(**d) for d in data.to_dict('records')]
logger.info("Starting")
result = store.add(records[:10])
logger.info("Done")

In [ ]:
store.load()

In [ ]:
store("Who is Hank Pym?")

In [ ]:
store("What can you tell me about captain america? What was his real name?")

### places

In [ ]:
import pandas as pd
data = pd.read_csv("/Users/sirsh/Downloads/nyc_zones.csv").drop(columns='id',index=1).rename(columns={'entity_key':'id'})
data['doc_id'] = data['id']
data['name'] = data['id']
data.head()

In [ ]:
store = VectorDataStore(Places)


In [ ]:
records = [Places(**d) for d in data.to_dict('records')]
store.add(records)

In [ ]:
store.load()

In [ ]:
store("What can you tell me about the civil airport in East Elmhurst Queens?")

In [ ]:
store.query_index("What can you tell me about civil airport in East Elmhurst Queens?")